In [172]:
import requests
import zipfile
from pathlib import Path

# Download and extract if needed
stanford_zip = Path("stanford-encyclopedia.zip")
stanford_folder = Path("stanford-encyclopedia")
if not stanford_zip.exists():
    zip_data = requests.get("https://github.com/MNoichl/data-driven-philosophy-GAP2025/raw/refs/heads/main/files/stanford-encyclopedia.zip").content
    with zipfile.ZipFile(zipfile.io.BytesIO(zip_data)) as zip_ref:
        zip_ref.extractall(".")

# Load data
text_data = [open(f, encoding='utf-8').read() for f in stanford_folder.glob("*.md")]
title_data = [f.stem for f in stanford_folder.glob("*.md")]
# Extract year data from copyright strings in the text files
import re

year_data = []
for text in text_data:
    # Look for "Copyright © YYYY" pattern at the beginning of the text
    match = re.search(r'Copyright © (\d{4})', text)  
    if match:
        year_data.append(int(match.group(1)))
    else:
        year_data.append(None)  # If no copyright year found

used_dataset = "Stanford Encyclopedia"

# Named Entity Recognition (NER)

Named entity recognition is a branch of natural language processing (NLP). Within an NLP pipeline, it is a relatively late step:

<center><img src='./files/NER_NLP_pipeline.png' width=700></center>

This is because it needs to receive a tokenized text and, in some languages, it needs to understand a word's part-of-speech (POS) to perform well. To determine whether a particular part of a text is a name, our model relies on a tokenization (splitting) of the text and its tagging for a particular part of speech (noun, verb, etc.). 

*(Graphic from William Mattingly, CC-BY, 2022 Text Analysis Pedagogy Institute)*

In [173]:
!python3 -m spacy download en_core_web_sm # for English NER

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [174]:
import spacy

In [175]:
nlp = spacy.load("en_core_web_sm")

Tokenization distinguishes full stops that end sentences from those that occur in abbreviations and initials (like G. E. Moore) - simply splitting by `.` would not give the right result.

In [176]:
text = "G.E. Moore is known as a good writer. Perhaps he is also good at programming?"

We can run the basic spacy pipeline on our text object like so:

In [177]:
doc = nlp(text)

The resulting doc object is structured. We can inspect its sentences:

In [178]:
list(doc.sents)

[G.E. Moore is known as a good writer.,
 Perhaps he is also good at programming?]

The spacy nlp pipeline also extracts a list of entities. Each entity has a text attribube (from the text source) and a label attribute, which we can inspect:

In [179]:
[(ent.text, ent.label_) for ent in doc.ents]

[('G.E. Moore', 'PERSON')]

This processing of the raw text into a doc object follows a number of steps that we can inspect (and adapt). 

In [180]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

## Looking at the Stanford Encyclopedia

Instead of looking at a toy sentence, we can process articles from the Stanford Encyclopedia. Here for a single (arbitrary) article:

In [181]:
doc = nlp(text_data[42])

The NER will now try to detect (and distinguish) persons, organizations, etc. If we only look at the entities tagged as persons:

In [182]:
[(ent.text, ent.label_) for ent in doc.ents if ent.label_=='PERSON'][0:20]

[("John Stuart Mill's", 'PERSON'),
 ('Mill 1859/1975', 'PERSON'),
 ('Schneewind', 'PERSON'),
 ('Wolff', 'PERSON'),
 ('Kant', 'PERSON'),
 ('Crocker 1980', 'PERSON'),
 ('Feinberg', 'PERSON'),
 ('Dworkin 1988', 'PERSON'),
 ('Feinberg', 'PERSON'),
 ('Dworkin 1989', 'PERSON'),
 ('Arneson', 'PERSON'),
 ('Raz 1986', 'PERSON'),
 ('Berofsky 1995', 'PERSON'),
 ('Raz 1986', 'PERSON'),
 ('Berofsky', 'PERSON'),
 ('Berofsky 1995', 'PERSON'),
 ('Veltman', 'PERSON'),
 ('Friedman', 'PERSON'),
 ('Kant', 'PERSON'),
 ('Hill', 'PERSON')]

In [183]:
print(f'We find {len([ent for ent in doc.ents if ent.label_=="PERSON"])} person labelled entities')

We find 79 person labelled entities


We can summarize this with a Counter object:

In [184]:
from collections import Counter
count = Counter()
count.update([ent.text for ent in doc.ents if ent.label_=='PERSON'])
count.most_common(10)

[('Kant', 9),
 ('Korsgaard', 4),
 ('Arneson', 3),
 ('Raz 1986', 3),
 ('Berofsky 1995', 3),
 ('Rawls', 3),
 ('Feinberg', 2),
 ('Benson', 2),
 ('Grovier 1993', 2),
 ('Taylor', 2)]

And clean some of the labels by removing the year-strings - this is also important to sum over variants.

In [185]:
strip_years("Kant's new add")

'Kants new add'

In [186]:
def strip_years(text):
    """
    Clean years and punctuation from labels.
    """
    return re.sub("\s?[0-9]|['|.)(]",'', text)

In [187]:
count = Counter()
count.update([strip_years(ent.text) for ent in doc.ents if ent.label_=='PERSON'])
count.most_common(10)

[('Kant', 9),
 ('Raz', 4),
 ('Berofsky', 4),
 ('Korsgaard', 4),
 ('Rawls', 4),
 ('Arneson', 3),
 ('Benson', 3),
 ('Feinberg', 2),
 ('Dworkin', 2),
 ('Grovier', 2)]

But have all person names been understood correctly? let's look at the Organizations.

In [188]:
count = Counter()
count.update([strip_years(ent.text) for ent in doc.ents if ent.label_=='ORG'])
count.most_common(10)

[('Stoljar', 5),
 ('Benson', 3),
 ('Kymlicka', 3),
 ('MacKenzie & Stoljar', 2),
 ('Christman', 2),
 ('Liberalism', 2),
 ('Cuypers', 1),
 ('MacCallum', 1),
 ('R Young', 1),
 ('Fisher & Ravizza', 1)]

A lot of these organization tags are actually referring to people - unsurprising, since many organizations are named after people. So we should expect some errors when trying to distinguish these automatically. 

In our specific case, we have background knowledge - we expect people to be mentioned much more than companies or organizations. And we know that many of these names are philosophers that should be tagged as persons. To take this into account, we can add an extra step in the NER pipeline.

## Adding an entity ruler

We can add explicit rules for detecting entities with an 'entity ruler'. This acts as an additional step in the spaCy pipeline.

In [189]:
names = ['Stoljar', 'Benson', 'Kymlicka'] # These can be many, feel free to add your own...
patterns = [{"label":"PERSON", "pattern":name} for name in names]

In [190]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [191]:
nlp2 = spacy.load("en_core_web_sm")

In [192]:
entruler = spacy.blank('en') # Create a blank English model
nlp_rules = nlp2.add_pipe("entity_ruler", before='ner') # specify that the EntityRuler comes before built-in NER
nlp_rules.add_patterns(patterns)

In [193]:
nlp2.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'entity_ruler',
 'ner']

The entity ruler now tags the patterns we have defined as persons, before we run the standard NER component. 

In [194]:
doc2 = nlp2(text_data[42])

In [195]:
count2 = Counter()
count2.update([strip_years(ent.text) for ent in doc2.ents if ent.label_=='PERSON'])
count2.most_common(10)

[('Kant', 9),
 ('Stoljar', 8),
 ('Benson', 7),
 ('Kymlicka', 7),
 ('Raz', 4),
 ('Berofsky', 4),
 ('Korsgaard', 4),
 ('Rawls', 4),
 ('Arneson', 3),
 ('Feinberg', 2)]

Compare again with the top names without our entity ruler:

In [196]:
doc = nlp(text_data[42])
count = Counter()
count.update([strip_years(ent.text) for ent in doc.ents if ent.label_=='PERSON'])
count.most_common(10)

[('Kant', 9),
 ('Raz', 4),
 ('Berofsky', 4),
 ('Korsgaard', 4),
 ('Rawls', 4),
 ('Arneson', 3),
 ('Benson', 3),
 ('Feinberg', 2),
 ('Dworkin', 2),
 ('Grovier', 2)]

Now the explicitly mentioned names are taken into account. Of course, in the raw text there are many more phrases and words that we might want to include as mentions of e.g. Kant:

In [197]:
c = Counter()
c.update([strip_years(word) for word in text_data[42].split(' ') if re.match('Kant', word)])

In [198]:
c

Counter({'Kantian': 12,
         'Kant': 5,
         'Kant,': 3,
         'Kants': 2,
         'Kantians': 1,
         'Kantianism': 1})

Is 'Kant' mentioned when Korsgaard is described as a Kantian? Even simple counting needs to answer some conceptual questions.

## Running on the full corpus
(not efficient, can take a while...) Here only taking the first 10

In [201]:
from tqdm import tqdm
docs = []
for text in tqdm(text_data[0:10]):
    docs.append(nlp2(text))

100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.63s/it]


In [202]:
corpcount = Counter()
for doc in docs:
    corpcount.update([strip_years(ent.text) for ent in doc.ents if ent.label_=='PERSON'])

In [203]:
corpcount.most_common()

[('Alexander', 175),
 ('Lucretius', 83),
 ('Reinach', 43),
 ('Plato', 39),
 ('Ibn Rushd', 26),
 ('Delmedigo', 21),
 ('Maimonides', 16),
 ('Alexander’s', 14),
 ('Mortensen', 10),
 ('Socrates', 9),
 ('Wodeham', 8),
 ('Ibn Rushds', 8),
 ('Ockham', 7),
 ('Aquinas', 6),
 ('Laird', 6),
 ('Crito', 6),
 ('Hohwy', 6),
 ('McKay et al', 6),
 ('Dunne', 5),
 ('Leff', 5),
 ('Aristotle', 5),
 ('John', 5),
 ('Libertarian Paternalism', 5),
 ('F H Bradley', 5),
 ('Alexanderii:', 5),
 ('Bortolotti', 5),
 ('Stephens', 5),
 ('Richard FitzRalph', 4),
 ('Wyclif', 4),
 ('Augustine', 4),
 ('God', 4),
 ('Kant', 4),
 ('Hegel', 4),
 ('Graham', 4),
 ('Currie', 4),
 ('Elijah Delmedigo', 4),
 ('Judaism', 4),
 ('Torah', 4),
 ('Weber', 4),
 ('Averroës', 3),
 ('Crathorn', 3),
 ('Memmius', 3),
 ('Empedocles', 3),
 ('Sunstein', 3),
 ('Samuel Alexander', 3),
 ('C Lloyd Morgan', 3),
 ('Emmet', 3),
 ('Jesus', 3),
 ('al', 3),
 ('Clouds', 3),
 ('Memorabilia', 3),
 ('Parmenides', 3),
 ('Xanthippe', 3),
 ('Kabbalah', 3),
 ('Bra